In [ ]:
''' 
0) Fix at_risk_value...right now CAD and USD positions are all being summed up. they need to be converted to the base currency first.
1) Calculate current amount_invested, portfolio value, current_leverage_multiplier, current liquidation value, current_return_pct, current_profit, at_risk_Value, at_risk_as_a_pct_of_current_liquidation_value
2) Run a strategy and figure out ideal portfolio
3) calculate the difference between the current portfolio and the ideal portfolio
4) calculate the delta for the portfolio and a list of orders to place.
5) Place the orders (with approvals of the user) (if the user says no, then the orders are not placed)
'''

In [1]:
import nest_asyncio
import json
from pprint import pprint
from ib_insync import IB, util, Stock, Order
nest_asyncio.apply()

In [ ]:
trader.disconnect()

In [2]:
class IBKRTrader:
    def __init__(self, update_stoplosses=False):
        # Connect to IBKR Trader Workstation
        self.ib = IB()
        self.connect()
        self.account_info = self.update_account_info()
        self.exchange_rate = self.get_exchange_rate()
        self.positions = self.current_positions()
        self.portfolio = self.current_portfolio()
        self.open_orders = self.get_open_orders()
        self.at_risk_portfolio = self.calculate_current_risk_portfolio()
        self.value_at_risk = sum(self.at_risk_portfolio.values())
        if update_stoplosses==True:
            self.update_stop_losses(risk_pct=0.10, max_acceptable_risk_deviation=0.2, force_update_stoplosses=False)
        pprint({'account_info': self.account_info,
            'value_at_risk': round(self.value_at_risk, 1),
            # 'Worst_case_scenario_NetLiquidation_value': int(self.account_info['NetLiquidation']) - self.value_at_risk
            })
        
    def connect(self):
        # Load the configuration file
        config = {'port':7777, 'host':'localhost'}
        # Connect to TWS
        self.ib.connect(config['host'], config['port'], clientId=1)
        print("Now Connecting to IBKR Trader Workstation.....")
    def disconnect(self):
        # Disconnect from TWS
        print("Now Disconnecting from IBKR Trader Workstation.....")
        self.ib.disconnect()
    def current_positions(self):
        # Get current positions
        self.positions = self.ib.positions()
        return self.positions
    def current_portfolio(self):
        # Get current portfolio value
        self.portfolio = self.ib.portfolio()
        return self.portfolio
    def get_open_orders(self):
        self.open_orders = self.ib.reqAllOpenOrders()
        return self.open_orders
    def calculate_current_risk_portfolio(self):
        self.portfolio = self.current_portfolio()
        self.open_orders = self.get_open_orders()
        at_risk_portfolio = {}
        for myposition in self.portfolio:
            symbol = myposition.contract.symbol
            primaryExchange = myposition.contract.primaryExchange
            currency = myposition.contract.currency
            stoploss_order = None
            for open_order in self.open_orders:
                if open_order.contract.symbol == symbol and open_order.order.orderType == 'STP':
                    stoploss_order = open_order
                    break
            
            if stoploss_order is None:
                market_value = myposition.position * myposition.marketPrice
                at_risk_portfolio[symbol] = market_value
            else:
                market_value = myposition.position * myposition.marketPrice
                stoploss_value = open_order.order.totalQuantity * open_order.order.auxPrice
                at_risk_value = market_value - stoploss_value
                at_risk_portfolio[symbol] = at_risk_value
                # print(f"Symbol: {symbol}, Market Value: {market_value}, Stop Loss Value: {stoploss_value}, At Risk Value: {at_risk_value}")
        
        self.at_risk_portfolio = at_risk_portfolio
        self.at_risk_value = sum(self.at_risk_portfolio.values())
        return self.at_risk_portfolio
    def place_stop_loss_order(self, direction, symbol, quantity, stop_price, primaryExchange, currency):
        # Define the contract for the symbol
        contract = Stock(symbol=symbol, exchange='SMART', currency=currency, primaryExchange=primaryExchange)

        # Create a stop-loss order
        stop_loss_order = Order(
            action=direction,           # 'SELL' to close or reduce a long position
            totalQuantity=quantity,     # Number of shares
            orderType='STP',            # Stop order
            auxPrice=stop_price,        # Stop price
            tif='GTC'                   # Good-till-canceled
        )

        # Place the order and return the order ID
        trade = self.ib.placeOrder(contract, stop_loss_order)
        self.ib.sleep(2)  # Wait for the order to be processed
        return trade.order.orderId
    def update_stop_losses(self, risk_pct=0.10, max_acceptable_risk_deviation=0.2, force_update_stoplosses=False):
        for myposition in self.portfolio:
            symbol = myposition.contract.symbol
            primaryExchange = myposition.contract.primaryExchange
            currency = myposition.contract.currency
            stoploss_order = None
            for open_order in self.open_orders:
                if open_order.contract.symbol == symbol and open_order.order.orderType == 'STP':
                    stoploss_order = open_order
                    break
            if stoploss_order is None:
                quantity = int(myposition.position)
                current_price = myposition.marketPrice
                stop_price = round(current_price * (1-risk_pct), 1)
                direction = 'SELL' if quantity > 0 else 'BUY'
                print(f"Placing a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, stop_price: {stop_price}, direction: {direction}")
                input("Press Enter to continue...")
                order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                print(f"Stop loss order placed with order ID: {order_id}")
            else:
                # check if stoploss is as per the risk profile.
                current_stoploss_price = stoploss_order.order.auxPrice
                current_stoploss_qty = int(stoploss_order.order.totalQuantity)
                current_position_qty = int(myposition.position)
                current_price = myposition.marketPrice
                direction = 'SELL' if current_position_qty > 0 else 'BUY'
                stoploss_pct_theshold = 1 - ((risk_pct * max_acceptable_risk_deviation) + risk_pct) if direction == 'SELL' else 1 + ((risk_pct * max_acceptable_risk_deviation) + risk_pct)
                stoploss_price_threshold = current_price * stoploss_pct_theshold
                
                if current_stoploss_qty != current_position_qty or force_update_stoplosses==True:
                    quantity = int(myposition.position)
                    current_price = myposition.marketPrice
                    stop_price = round(current_price * (1-risk_pct), 1)
                    print(f"Updating a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, stop_price: {stop_price}, direction: {direction}")
                    input("Press Enter to continue...")
                    order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                    print(f"Stop loss order updated with order ID: {order_id}")
                    self.ib.cancelOrder(stoploss_order.order)
                    print(f"Previous Stop loss order with order ID: {stoploss_order.order.orderId} is cancelled.")

                elif (direction == 'SELL' and current_stoploss_price < stoploss_price_threshold) or (direction == 'BUY' and current_stoploss_price > stoploss_price_threshold):
                    quantity = int(myposition.position)
                    current_price = myposition.marketPrice
                    stop_price = round(current_price * (1-risk_pct), 1)
                    print(f"Updating a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, old_stop_price: {current_stoploss_price}, new_stop_price: {stop_price}, direction: {direction}")
                    input("Press Enter to continue...")
                    order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                    print(f"Stop loss order updated with order ID: {order_id}")
                    self.ib.cancelOrder(stoploss_order.order)
                    print(f"Previous Stop loss order with order ID: {stoploss_order.order.orderId} is cancelled.")
                else:
                    # if force_update_stoplosses==True:
                    print(f"Stop loss order for symbol: {symbol} is already at an acceptable level.")
                    
        # update open orders
        self.open_orders = self.get_open_orders()
        # update at_risk_portfolio
        self.at_risk_portfolio = self.calculate_current_risk_portfolio()
        self.at_risk_value = sum(self.at_risk_portfolio.values())
    def update_account_info(self):
        account_summary = self.ib.accountSummary()
        account_info = {}
        for item in account_summary:
            if item.tag == 'NetLiquidation':
                account_info['NetLiquidation'] = str(item.currency) + ' ' + str(item.value)
            if item.tag == 'GrossPositionValue':
                account_info['GrossPositionValue'] = str(item.currency) + ' ' + str(item.value)
        account_info['current_leverage_multiplier'] = round(float(account_info['GrossPositionValue'].split(' ')[1]) / float(account_info['NetLiquidation'].split(' ')[1]), 2)
            
        self.account_info = account_info
        return self.account_info
    def get_exchange_rate(self):
        account_summary = self.ib.accountSummary()
        for item in account_summary:
            if item.tag == 'ExchangeRate':
                self.exchange_rate = float(item.value)
                break

In [ ]:
# Create an instance of the IBKRTrader class
trader = IBKRTrader(update_stoplosses=False)

In [ ]:
trader.update_stop_losses(risk_pct=0.075, max_acceptable_risk_deviation=0.1, force_update_stoplosses=False)

In [ ]:
rate = trader.get_exchange_rate()
rate

In [ ]:
positions = trader.current_positions()
portfolio = trader.current_portfolio()
open_orders = trader.get_open_orders()

In [ ]:
portfolio

In [ ]:
open_orders

In [2]:
# load /Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/junk/old_code/stock_symbols.pkl
import pickle
with open('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/junk/old_code/stock_symbols.pkl', 'rb') as f:
    stock_symbols = pickle.load(f)

In [7]:
for symbol in stock_symbols:
    if type(symbol) == float:
        stock_symbols.remove(symbol)

In [8]:
# sor the list
stock_symbols = sorted(stock_symbols)

# save list to a txt file
with open('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/junk/old_code/stock_symbols.txt', 'w') as f:
    for item in stock_symbols:
        f.write("%s\n" % item)

In [14]:
stock_symbols

['A',
 'AA',
 'AACG',
 'AACI',
 'AACIW',
 'AACT',
 'AADI',
 'AAGR',
 'AAGRW',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AB',
 'ABAT',
 'ABBV',
 'ABCB',
 'ABCL',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABL',
 'ABLLW',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABSI',
 'ABT',
 'ABTS',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACB',
 'ACBA',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACEL',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIC',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACP',
 'ACR',
 'ACRE',
 'ACRS',
 'ACRV',
 'ACST',
 'ACT',
 'ACTG',
 'ACU',
 'ACV',
 'ACVA',
 'ACXP',
 'ADAG',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADD',
 'ADEA',
 'ADI',
 'ADIL',
 'ADM',
 'ADMA',
 'ADN',
 'ADNT',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADRT',
 'ADSE',
 'ADSEW',
 'ADSK',
 'ADT',
 'ADTH',
 'ADTHW',
 'A

In [57]:
from datetime import datetime, time, timedelta
import pytz

def is_market_open(current_datetime):
    # Define market open and close times
    MARKET_OPEN_TIME = time(9, 30)  # 9:30 AM
    MARKET_CLOSE_TIME = time(16, 0)  # 4:00 PM
    """
    Check if the market is open at the given datetime.
    
    Parameters:
    current_datetime (datetime): The datetime to check.
    
    Returns:
    bool: True if the market is open, False otherwise.
    """
    current_time = current_datetime.time()
    return MARKET_OPEN_TIME <= current_time <= MARKET_CLOSE_TIME

def next_market_open(current_datetime):
    # Define market open and close times
    MARKET_OPEN_TIME = time(9, 30)  # 9:30 AM
    MARKET_CLOSE_TIME = time(16, 0)  # 4:00 PM
    """
    Get the next market open datetime from the given datetime.
    
    Parameters:
    current_datetime (datetime): The datetime to check from.
    
    Returns:
    datetime: The next market open datetime.
    """
    current_time = current_datetime.time()
    if current_time < MARKET_OPEN_TIME:
        # If current time is before market opens today
        next_open = current_datetime.replace(hour=MARKET_OPEN_TIME.hour, minute=MARKET_OPEN_TIME.minute, second=0, microsecond=0)
    elif current_time > MARKET_CLOSE_TIME:
        # If current time is after market closes today
        next_open = (current_datetime + timedelta(days=1)).replace(hour=MARKET_OPEN_TIME.hour, minute=MARKET_OPEN_TIME.minute, second=0, microsecond=0)
    else:
        # If current time is during market hours
        next_open = current_datetime.replace(hour=MARKET_OPEN_TIME.hour, minute=MARKET_OPEN_TIME.minute, second=0, microsecond=0)
    
    return next_open

In [58]:
def convert_weekday_num_to_string(weekday_num):
    weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    return weekdays[weekday_num]

In [72]:
# Example usage
current_time = datetime(2023, 9, 17, 9, 29, 0, 0, pytz.timezone('US/Eastern'))
print('Current_time weekday:', convert_weekday_num_to_string(current_time.weekday()))
# Example usage

if is_market_open(current_time):
    print("The market is open.")
else:
    print("The market is closed.")
    print(f"The next market open is on {convert_weekday_num_to_string(next_market_open(current_time).weekday())}, {next_market_open(current_time).strftime('%Y-%m-%d %H:%M:%S %Z')}")
    
    # calculate the number of seconds until the next market open
    next_open = next_market_open(current_time)
    secs_until_next_open = (next_open - current_time).total_seconds()
    print(f"Time until next market open: {secs_until_next_open} seconds")

Current_time weekday: Sunday
The market is closed.
The next market open is on Sunday, 2023-09-17 09:30:00 LMT
Time until next market open: 60.0 seconds


In [26]:
current_time

datetime.datetime(2023, 9, 15, 14, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>)